In [1]:
import ast
import csv
import io
import os
import json as js
import pickle
import sys
import tempfile
from collections import Counter
import torch
import gdown
import numpy as np
import torchvision
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from ExtractEmbedding import QueryToEmbedding, QueryToEmbeddingiNat
from SimSearch import FaissCosineNeighbors, SearchableTrainingSet
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Caluclate Accuracy
with open(f"./embeddings_iNat.pickle", "rb") as f:
    Xtrain = pickle.load(f)

with open(f"./labels.pickle", "rb") as f:
    ytrain = pickle.load(f)


In [3]:

searcher = SearchableTrainingSet(Xtrain, ytrain)
searcher.build_index()

# Extract label names
training_folder = ImageFolder(root="./data/train/")

id_to_bird_name = {
    x[1]: x[0].split("/")[-2].replace(".", " ") for x in training_folder.imgs
}

def knn_get_neighbors(query, k=20):
    query_embedding = QueryToEmbeddingiNat(query)
    scores, indices, labels = searcher.search(query_embedding, k=k)
    return scores, indices, labels


In [4]:
test_folder = ImageFolder(root="./data/test/")

In [5]:
knn_results = {}

for i in tqdm(range(len(test_folder))):
    query = test_folder.imgs[i][0]
    scores, indices, labels = knn_get_neighbors(query, k=50)
    knn_results[query] = {
        "scores": scores,
        "indices": indices,
        "labels": labels,
    }

100%|██████████| 5794/5794 [53:05<00:00,  1.82it/s]


In [7]:
# save results in a pickel format

with open(f"./knn_results.pickle", "wb") as f:
    pickle.dump(knn_results, f)

### Analysis

In [17]:
# save results in a pickel format

with open(f"./knn_results.pickle", "rb") as f:
    knn_results = pickle.load(f)

In [18]:
gt_labels = [x[1] for x in test_folder.imgs]

In [33]:
# for K=50
knn50_preds = [Counter(knn_results[x[0]]["labels"][0]).most_common(1)[0][0] for x in test_folder.imgs]
# calcualte accuracy
knn50_acc = np.mean(np.array(gt_labels) == np.array(knn50_preds))
print(f"KNN Accuracy: {100*knn50_acc:0.2f}")

KNN Accuracy: 82.81


In [34]:
knn20_preds = [Counter(knn_results[x[0]]["labels"][0][:20]).most_common(1)[0][0] for x in test_folder.imgs]
knn20_acc = np.mean(np.array(gt_labels) == np.array(knn20_preds))
print(f"KNN Accuracy: {100*knn20_acc:0.2f}")

KNN Accuracy: 85.88


In [44]:
knn_50_good = []
for i in range(len(knn50_preds)):
    if knn50_preds[i] == gt_labels[i] and knn20_preds[i] != gt_labels[i]:
        knn_50_good.append(test_folder.imgs[i][0])
        

In [48]:
# shuffle
knn_50_good = np.random.permutation(knn_50_good)
knn_50_good

array(['./data/test/145.Elegant_Tern/Elegant_Tern_0033_150687.jpg',
       './data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0030_796994.jpg',
       './data/test/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0046_118032.jpg',
       './data/test/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0085_795167.jpg',
       './data/test/102.Western_Wood_Pewee/Western_Wood_Pewee_0050_98241.jpg',
       './data/test/082.Ringed_Kingfisher/Ringed_Kingfisher_0033_73133.jpg',
       './data/test/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0088_168052.jpg',
       './data/test/030.Fish_Crow/Fish_Crow_0012_25946.jpg',
       './data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0010_796964.jpg',
       './data/test/177.Prothonotary_Warbler/Prothonotary_Warbler_0100_174539.jpg',
       './data/test/043.Yellow_bellied_Flycatcher/Yellow_Bellied_Flycatcher_0060_42595.jpg',
       './data/test/154.Red_eyed_Vireo/Red_Eyed_Vireo_0074_157170.jpg',
       './data/test/009.Brewer_Blackbird/Brewer_Blac

In [ ]:
array(['./data/test/145.Elegant_Tern/Elegant_Tern_0033_150687.jpg',
       './data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0030_796994.jpg',
       './data/test/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0046_118032.jpg',
       './data/test/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0085_795167.jpg',
       './data/test/102.Western_Wood_Pewee/Western_Wood_Pewee_0050_98241.jpg',
       './data/test/082.Ringed_Kingfisher/Ringed_Kingfisher_0033_73133.jpg',
       './data/test/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0088_168052.jpg',
       './data/test/030.Fish_Crow/Fish_Crow_0012_25946.jpg',
       './data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0010_796964.jpg',
       './data/test/177.Prothonotary_Warbler/Prothonotary_Warbler_0100_174539.jpg',
       './data/test/043.Yellow_bellied_Flycatcher/Yellow_Bellied_Flycatcher_0060_42595.jpg',
       './data/test/154.Red_eyed_Vireo/Red_Eyed_Vireo_0074_157170.jpg',
       './data/test/009.Brewer_Blackbird/Brewer_Blackbird_0017_2668.jpg',
       './data/test/049.Boat_tailed_Grackle/Boat_Tailed_Grackle_0116_33808.jpg',
       './data/test/091.Mockingbird/Mockingbird_0018_81183.jpg',
       './data/test/029.American_Crow/American_Crow_0085_25260.jpg',
       './data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0064_795584.jpg',
       './data/test/111.Loggerhead_Shrike/Loggerhead_Shrike_0028_106221.jpg',
       './data/test/153.Philadelphia_Vireo/Philadelphia_Vireo_0082_156574.jpg',
       './data/test/039.Least_Flycatcher/Least_Flycatcher_0085_30318.jpg',
       './data/test/187.American_Three_toed_Woodpecker/American_Three_Toed_Woodpecker_0032_796170.jpg',
       './data/test/096.Hooded_Oriole/Hooded_Oriole_0088_90356.jpg',
       './data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0011_796954.jpg',
       './data/test/027.Shiny_Cowbird/Shiny_Cowbird_0059_24421.jpg',
       './data/test/091.Mockingbird/Mockingbird_0060_79972.jpg',
       './data/test/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0013_795176.jpg',
       './data/test/027.Shiny_Cowbird/Shiny_Cowbird_0085_796839.jpg',
       './data/test/060.Glaucous_winged_Gull/Glaucous_Winged_Gull_0035_44576.jpg',
       './data/test/178.Swainson_Warbler/Swainson_Warbler_0053_174678.jpg',
       './data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0052_795610.jpg',
       './data/test/072.Pomarine_Jaeger/Pomarine_Jaeger_0023_61431.jpg',
       './data/test/041.Scissor_tailed_Flycatcher/Scissor_Tailed_Flycatcher_0125_41906.jpg',
       './data/test/183.Northern_Waterthrush/Northern_Waterthrush_0062_177364.jpg',
       './data/test/023.Brandt_Cormorant/Brandt_Cormorant_0064_22849.jpg',
       './data/test/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0027_118028.jpg',
       './data/test/062.Herring_Gull/Herring_Gull_0037_48655.jpg',
       './data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0022_29145.jpg',
       './data/test/066.Western_Gull/Western_Gull_0012_53605.jpg',
       './data/test/105.Whip_poor_Will/Whip_Poor_Will_0036_100399.jpg',
       './data/test/112.Great_Grey_Shrike/Great_Grey_Shrike_0075_797022.jpg',
       './data/test/144.Common_Tern/Common_Tern_0115_149482.jpg',
       './data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0051_29077.jpg',
       './data/test/029.American_Crow/American_Crow_0079_25463.jpg',
       './data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0039_797000.jpg',
       './data/test/060.Glaucous_winged_Gull/Glaucous_Winged_Gull_0013_44381.jpg',
       './data/test/032.Mangrove_Cuckoo/Mangrove_Cuckoo_0026_794609.jpg',
       './data/test/026.Bronzed_Cowbird/Bronzed_Cowbird_0064_24199.jpg',
       './data/test/043.Yellow_bellied_Flycatcher/Yellow_Bellied_Flycatcher_0013_42769.jpg',
       './data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0072_795603.jpg',
       './data/test/064.Ring_billed_Gull/Ring_Billed_Gull_0007_51430.jpg',
       './data/test/119.Field_Sparrow/Field_Sparrow_0092_113580.jpg',
       './data/test/111.Loggerhead_Shrike/Loggerhead_Shrike_0091_105076.jpg',
       './data/test/112.Great_Grey_Shrike/Great_Grey_Shrike_0055_106858.jpg',
       './data/test/198.Rock_Wren/Rock_Wren_0067_189152.jpg',
       './data/test/123.Henslow_Sparrow/Henslow_Sparrow_0073_116803.jpg',
       './data/test/098.Scott_Oriole/Scott_Oriole_0055_795825.jpg',
       './data/test/035.Purple_Finch/Purple_Finch_0130_27555.jpg',
       './data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0060_795604.jpg',
       './data/test/050.Eared_Grebe/Eared_Grebe_0018_34357.jpg',
       './data/test/108.White_necked_Raven/White_Necked_Raven_0009_102518.jpg',
       './data/test/132.White_crowned_Sparrow/White_Crowned_Sparrow_0085_127194.jpg',
       './data/test/067.Anna_Hummingbird/Anna_Hummingbird_0126_55983.jpg',
       './data/test/078.Gray_Kingbird/Gray_Kingbird_0051_70139.jpg',
       './data/test/155.Warbling_Vireo/Warbling_Vireo_0072_158422.jpg',
       './data/test/144.Common_Tern/Common_Tern_0088_147941.jpg',
       './data/test/063.Ivory_Gull/Ivory_Gull_0073_49287.jpg',
       './data/test/045.Northern_Fulmar/Northern_Fulmar_0049_43906.jpg',
       './data/test/145.Elegant_Tern/Elegant_Tern_0023_150654.jpg',
       './data/test/067.Anna_Hummingbird/Anna_Hummingbird_0101_56459.jpg',
       './data/test/112.Great_Grey_Shrike/Great_Grey_Shrike_0086_106533.jpg',
       './data/test/145.Elegant_Tern/Elegant_Tern_0084_150922.jpg',
       './data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0042_29127.jpg',
       './data/test/023.Brandt_Cormorant/Brandt_Cormorant_0071_23007.jpg',
       './data/test/146.Forsters_Tern/Forsters_Tern_0042_151644.jpg',
       './data/test/127.Savannah_Sparrow/Savannah_Sparrow_0075_116305.jpg',
       './data/test/114.Black_throated_Sparrow/Black_Throated_Sparrow_0057_107036.jpg',
       './data/test/113.Baird_Sparrow/Baird_Sparrow_0017_794586.jpg',
       './data/test/123.Henslow_Sparrow/Henslow_Sparrow_0086_796545.jpg',
       './data/test/144.Common_Tern/Common_Tern_0045_150067.jpg',
       './data/test/060.Glaucous_winged_Gull/Glaucous_Winged_Gull_0056_44658.jpg',
       './data/test/096.Hooded_Oriole/Hooded_Oriole_0041_90218.jpg',
       './data/test/105.Whip_poor_Will/Whip_Poor_Will_0021_796426.jpg',
       './data/test/043.Yellow_bellied_Flycatcher/Yellow_Bellied_Flycatcher_0039_795471.jpg',
       './data/test/115.Brewer_Sparrow/Brewer_Sparrow_0034_796693.jpg'],
      dtype='<U93')

In [ ]:
['./data/test/145.Elegant_Tern/Elegant_Tern_0033_150687.jpg'],
['./data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0030_796994.jpg'],
['./data/test/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0046_118032.jpg'],
['./data/test/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0085_795167.jpg'],
['./data/test/102.Western_Wood_Pewee/Western_Wood_Pewee_0050_98241.jpg'],
['./data/test/082.Ringed_Kingfisher/Ringed_Kingfisher_0033_73133.jpg'],
['./data/test/173.Orange_crowned_Warbler/Orange_Crowned_Warbler_0088_168052.jpg'],
['./data/test/030.Fish_Crow/Fish_Crow_0012_25946.jpg'],
['./data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0010_796964.jpg'],
['./data/test/177.Prothonotary_Warbler/Prothonotary_Warbler_0100_174539.jpg'],
['./data/test/043.Yellow_bellied_Flycatcher/Yellow_Bellied_Flycatcher_0060_42595.jpg'],
['./data/test/154.Red_eyed_Vireo/Red_Eyed_Vireo_0074_157170.jpg'],
['./data/test/009.Brewer_Blackbird/Brewer_Blackbird_0017_2668.jpg'],
['./data/test/049.Boat_tailed_Grackle/Boat_Tailed_Grackle_0116_33808.jpg'],
['./data/test/091.Mockingbird/Mockingbird_0018_81183.jpg'],
['./data/test/029.American_Crow/American_Crow_0085_25260.jpg'],
['./data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0064_795584.jpg'],
['./data/test/111.Loggerhead_Shrike/Loggerhead_Shrike_0028_106221.jpg'],
['./data/test/153.Philadelphia_Vireo/Philadelphia_Vireo_0082_156574.jpg'],
['./data/test/039.Least_Flycatcher/Least_Flycatcher_0085_30318.jpg'],
['./data/test/187.American_Three_toed_Woodpecker/American_Three_Toed_Woodpecker_0032_796170.jpg'],
['./data/test/096.Hooded_Oriole/Hooded_Oriole_0088_90356.jpg'],
['./data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0011_796954.jpg'],
['./data/test/027.Shiny_Cowbird/Shiny_Cowbird_0059_24421.jpg'],
['./data/test/091.Mockingbird/Mockingbird_0060_79972.jpg'],
['./data/test/124.Le_Conte_Sparrow/Le_Conte_Sparrow_0013_795176.jpg'],
['./data/test/027.Shiny_Cowbird/Shiny_Cowbird_0085_796839.jpg'],
['./data/test/060.Glaucous_winged_Gull/Glaucous_Winged_Gull_0035_44576.jpg'],
['./data/test/178.Swainson_Warbler/Swainson_Warbler_0053_174678.jpg'],
['./data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0052_795610.jpg'],
['./data/test/072.Pomarine_Jaeger/Pomarine_Jaeger_0023_61431.jpg'],
['./data/test/041.Scissor_tailed_Flycatcher/Scissor_Tailed_Flycatcher_0125_41906.jpg'],
['./data/test/183.Northern_Waterthrush/Northern_Waterthrush_0062_177364.jpg'],
['./data/test/023.Brandt_Cormorant/Brandt_Cormorant_0064_22849.jpg'],
['./data/test/126.Nelson_Sharp_tailed_Sparrow/Nelson_Sharp_Tailed_Sparrow_0027_118028.jpg'],
['./data/test/062.Herring_Gull/Herring_Gull_0037_48655.jpg'],
['./data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0022_29145.jpg'],
['./data/test/066.Western_Gull/Western_Gull_0012_53605.jpg'],
['./data/test/105.Whip_poor_Will/Whip_Poor_Will_0036_100399.jpg'],
['./data/test/112.Great_Grey_Shrike/Great_Grey_Shrike_0075_797022.jpg'],
['./data/test/144.Common_Tern/Common_Tern_0115_149482.jpg'],
['./data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0051_29077.jpg'],
['./data/test/029.American_Crow/American_Crow_0079_25463.jpg'],
['./data/test/022.Chuck_will_Widow/Chuck_Will_Widow_0039_797000.jpg'],
['./data/test/060.Glaucous_winged_Gull/Glaucous_Winged_Gull_0013_44381.jpg'],
['./data/test/032.Mangrove_Cuckoo/Mangrove_Cuckoo_0026_794609.jpg'],
['./data/test/026.Bronzed_Cowbird/Bronzed_Cowbird_0064_24199.jpg'],
['./data/test/043.Yellow_bellied_Flycatcher/Yellow_Bellied_Flycatcher_0013_42769.jpg'],
['./data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0072_795603.jpg'],
['./data/test/064.Ring_billed_Gull/Ring_Billed_Gull_0007_51430.jpg'],
['./data/test/119.Field_Sparrow/Field_Sparrow_0092_113580.jpg'],
['./data/test/111.Loggerhead_Shrike/Loggerhead_Shrike_0091_105076.jpg'],
['./data/test/112.Great_Grey_Shrike/Great_Grey_Shrike_0055_106858.jpg'],
['./data/test/198.Rock_Wren/Rock_Wren_0067_189152.jpg'],
['./data/test/123.Henslow_Sparrow/Henslow_Sparrow_0073_116803.jpg'],
['./data/test/098.Scott_Oriole/Scott_Oriole_0055_795825.jpg'],
['./data/test/035.Purple_Finch/Purple_Finch_0130_27555.jpg'],
['./data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0060_795604.jpg'],
['./data/test/050.Eared_Grebe/Eared_Grebe_0018_34357.jpg'],
['./data/test/108.White_necked_Raven/White_Necked_Raven_0009_102518.jpg'],
['./data/test/132.White_crowned_Sparrow/White_Crowned_Sparrow_0085_127194.jpg'],
['./data/test/067.Anna_Hummingbird/Anna_Hummingbird_0126_55983.jpg'],
['./data/test/078.Gray_Kingbird/Gray_Kingbird_0051_70139.jpg'],
['./data/test/155.Warbling_Vireo/Warbling_Vireo_0072_158422.jpg'],
['./data/test/144.Common_Tern/Common_Tern_0088_147941.jpg'],
['./data/test/063.Ivory_Gull/Ivory_Gull_0073_49287.jpg'],
['./data/test/045.Northern_Fulmar/Northern_Fulmar_0049_43906.jpg'],
['./data/test/145.Elegant_Tern/Elegant_Tern_0023_150654.jpg'],
['./data/test/067.Anna_Hummingbird/Anna_Hummingbird_0101_56459.jpg'],
['./data/test/112.Great_Grey_Shrike/Great_Grey_Shrike_0086_106533.jpg'],
['./data/test/145.Elegant_Tern/Elegant_Tern_0084_150922.jpg'],
['./data/test/037.Acadian_Flycatcher/Acadian_Flycatcher_0042_29127.jpg'],
['./data/test/023.Brandt_Cormorant/Brandt_Cormorant_0071_23007.jpg'],
['./data/test/146.Forsters_Tern/Forsters_Tern_0042_151644.jpg'],
['./data/test/127.Savannah_Sparrow/Savannah_Sparrow_0075_116305.jpg'],
['./data/test/114.Black_throated_Sparrow/Black_Throated_Sparrow_0057_107036.jpg'],
['./data/test/113.Baird_Sparrow/Baird_Sparrow_0017_794586.jpg'],
['./data/test/123.Henslow_Sparrow/Henslow_Sparrow_0086_796545.jpg'],
['./data/test/144.Common_Tern/Common_Tern_0045_150067.jpg'],
['./data/test/060.Glaucous_winged_Gull/Glaucous_Winged_Gull_0056_44658.jpg'],
['./data/test/096.Hooded_Oriole/Hooded_Oriole_0041_90218.jpg'],
['./data/test/105.Whip_poor_Will/Whip_Poor_Will_0021_796426.jpg'],
['./data/test/043.Yellow_bellied_Flycatcher/Yellow_Bellied_Flycatcher_0039_795471.jpg'],
['./data/test/115.Brewer_Sparrow/Brewer_Sparrow_0034_796693.jpg]'